# LIBRARIES

In [1]:
import Learner
from Environment import Environment
from UserCat import UserCat
from Product import Product
import numpy as np

In [10]:
users=[]
products=[]

nameofproduct= [ #name of products
    "Calabazas",
    "Hinojo",
    "Sesamo",
    "Girasol",
    "Amapola"
]

prices=[[4., 5, 6 ,7],
    [9., 10, 11, 12],
    [18., 19, 20, 21],
    [24., 25, 26, 27],
    [33., 34, 35, 36]]
#1-2 di delta, Con sovrapposizione

cost=[2,4.5,9,14,17]

#sarebbe interessante anche prendere da file il tutto così da cambiare tutto più facilmente
#calcolo i margini dai cost mi sembra più sensato e anche più veloce se dobbiamo cambiare continuamente

cost2 = np.tile(np.array([cost]).transpose(), (1, 4))
margins = np.array(prices)-cost2

Secondary_dict={           # Propongo i prodotti più simili a quello mostrato --> problemino: 2 viene mostrato quasi sempre
    "Calabazas": [1,2],
    "Hinojo": [0,2],
    "Sesamo": [1,3],
    "Girasol": [2,4],
    "Amapola": [2,3]
}

res_price_params={
    "shape": 15,
    "scale": 2,
}

probabilities = np.random.uniform(0.0,1,(5, 5))

#matrix generata da una uniform / 5 because it is the number of product --> Andre: 0.1 come max non è un po' poco?

alphas=[1/5, 1/5, 1/5, 1/5, 1/5] 
# per ora li generiamo così, tutti uguali -> devo generare 3 diversi vettori alpha

poisson_lambda = 1
#=valore atteso del numero di prodotti acquistati (specifico per prodotto)...non dipende dal
#prodotto oltre che dallo user che dal tipo di user che

p_users=[4/9, 3/9, 2/9] #probabilità di essere un tipo di utente-> da cambiare

lambda_q = 0.5 #just my idea of lambda
#possiamo stimarlo con i dati passati provenienti dal sito -> vino tot è stato comprato 15 volte

#proviamo a pensare, ha senso vederlo come coppia? categoria-prodotto? Avrei 3 categorie *5 prodotti-> 15 lambda diversi
# ^^ Andre: secondo me ha senso avere 3 lambda_q diversi per categoria MA non per prodotto. La probabilità di continuare
#           a guardare secondo me dipende dall'utente e non dal prodotto

for i in range (5):
    products.append(Product(prices[i], i, nameofproduct[i],margins[i]))

for i in range(3):
    users.append(UserCat(alphas, res_price_params, poisson_lambda, probabilities))

Env = Environment(users, products,  lambda_q, Secondary_dict,p_users)


In [14]:
Env.aggregated_reward([0, 0, 0, 0, 0])

[<Product.Product object at 0x14f41f490>] [0]
[<Product.Product object at 0x14f41f490>, <Product.Product object at 0x12f95ae90>] [0, 0.13001219807043696]
[<Product.Product object at 0x14f41f490>, <Product.Product object at 0x12f95ae90>, <Product.Product object at 0x12f95b970>] [0, 0.13001219807043696, 0.32616595008093935]
[<Product.Product object at 0x14f41f490>] [0]
[<Product.Product object at 0x14f41f490>, <Product.Product object at 0x12f95ae90>] [0, 0.21587204719900477]
[<Product.Product object at 0x14f41f490>, <Product.Product object at 0x12f95ae90>, <Product.Product object at 0x12f95b970>] [0, 0.21587204719900477, 0.32616595008093935]
[<Product.Product object at 0x14f41f490>] [0]
[<Product.Product object at 0x14f41f490>, <Product.Product object at 0x12f95b970>] [0, 0.32616595008093935]
[<Product.Product object at 0x14f41f490>] [0]
[<Product.Product object at 0x14f41f490>, <Product.Product object at 0x12f95b640>] [0, 0.1692760981437984]
[<Product.Product object at 0x14f41f490>] [0]

2.9558640364966764

In [4]:
Env.Secondary_dict[products[1].name]

[0, 2]

In [6]:
a = [1, 2, 3, 4, 5]
a[-1]
a[:-1]

[1, 2, 3, 4]

In [4]:
a = [0]
a[:-1]

[]

In [5]:
print(a[:-1])

[]
